# Parser FINAL

Importamo potrebne knjižnice.

In [1]:
import nltk
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
import time

Spodaj je funkcija, ki prejme url in vrne sprasan naslov, telo in komentarje novice.

In [2]:
def parse_url(url):
    try:
        driver = webdriver.Chrome('..\\webdriver\\chromedriver.exe')
        driver.get(url)
        button = driver.find_element_by_class_name('comment-show-hide')
        #webdriver.ActionChains(driver).move_to_element(button).click(button).perform()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        button.click()
        time.sleep(1)
        html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
        #html = driver.page_source
        #print(html)
        driver.close()
        
        soup = BeautifulSoup(html, 'html.parser')
        newsContents = soup.find_all("div", id="newsbody")[0].find_all("p")
        parsed = ""
        
        #print(html)
        
        comments = soup.find_all("div", class_="newscomments")[0].find_all("dt", class_="ds2")
        #print("QWE: " + comments.get_text())
        for p in newsContents: 
            parsed += p.get_text() + " "
        title = soup.find_all("h1")[0].get_text()
        parsed_comments = [c.get_text() for c in comments]
        return parsed, title, parsed_comments
    except:
        print("Error url: " + url)
        return "", "", ""

Test klica funkcije.

In [ ]:
parse_url("http://www.rtvslo.si/svet/bliznji-vzhod/smrt-ameriki-iranci-protestirajo-proti-novim-sankcijam/470795")

Naredimo še funkcijo, ki s pomočjo Rtvslo iskalnika vrne vse povezave do najdenih novic.

In [3]:
def get_urls(query, num_of_pages=1):
    urls = []
    for i in range(num_of_pages):
        html = urlopen("http://www.rtvslo.si/iskalnik?q=" + query + "&page=" + str(i)).read() 
        soup = BeautifulSoup(html, 'html.parser')
        for news in soup.find_all("div", class_="stitle"):
            urls.append("http://www.rtvslo.si" + news.a["href"])
    return urls

In [ ]:
get_urls("trump", 2)

Definiramo seznam novic, ki bi jih radi sparsali.

Za vsako novico kličemo prvo metodo.

In [13]:
urls = get_urls("doncic", 1)
parsed_news = []
for url in urls:
    parsed, title, comments = parse_url(url)
    if parsed != "":
        parsed_news.append((title, parsed, comments))
parsed_news

Error url: http://www.rtvslo.si/news-in-english/doncic-i-can-t-believe-i-m-in-the-same-team-with-the-legendary-nowitzki/466709
Error url: http://www.rtvslo.si/radio-si/news/doncic-to-go-to-dallas/458736
Error url: http://www.rtvslo.si/news-in-english/king-luka-conquers-basketball-europe/455637
Error url: http://www.rtvslo.si/news-in-english/from-istanbul-to-belgrade-doncic-only-one-more-victory-away-from-perfect-season/455502
Error url: http://www.rtvslo.si/news-in-english/no-mvp-titles-for-doncic-without-wins-for-real/440253
Error url: http://www.rtvslo.si/news-in-english/photo-luka-doncic-s-heart-beats-for-slovenia-and-his-beautiful-anamarija/432690
Error url: http://www.rtvslo.si/zabava/druzabna-kronika/foto-srce-luke-doncica-bije-za-slovenijo-in-za-lepo-anamarijo/432675
Error url: http://www.rtvslo.si/news-in-english/basketball-sharp-shooting-slovenia-shock-spain-in-eurobasket-semi/432597


[('Dončić po desetini sezone: ali pri Teličkih manjka predvsem kemija?',
  'Jesenska priljubljena tema športnih pogovorov so krstne tekme Luka Dončića v Ligi NBA. Presežnikov in razprav ne manjka, kar je pripeljalo do 5. oddaje podkasta SOS-odmev, kjer je LD77 osrednja tema pogovora. Za boljši strokovni uvid je svoje misli prispeval MMC-jev specialist za košarko Tilen Jamnik, ki že leta podrobno spremlja kariero Luke Dončića in dobro pozna tudi razmišljanje ter delovanje Lige NBA. Družbo je delal obema gostiteljema podkasta SOS-odmev Slavku Jeriču in Toniju Grudnu.  Naročite se na podkast SOS-odmev! Z menjavo očitno zadela oba klubaKoliko je vreden MVP Evrolige? Očitno toliko kot najboljši strelec in podajalec študentske lige NCAA ter še dodatni igralec, toliko je namreč bila vrednost menjave med Atlanto in Dallasom, ki sta si ob poletnem izboru Lige NBA ekspresno medsebojno zamenjala dva izredno nadarjena košarkarja. To sta Trae Young in Luka Dončić. "Vidi se, kako si je Dallas res že

Zapišemo v datoteko.

In [14]:
def write_to_file():
    f = open("../data/novice_doncic.txt", "w", encoding="utf-8")
    for title, parsed, comments in parsed_news:
        f.write(title + "\n")
        f.write(parsed + "\n")
        for c in comments:
            if c.replace("\n", "") == "":
                continue
            f.write(c.replace("\n", " ") + "\n")
        f.write("\n")

In [15]:
write_to_file()